In [1]:
import tensorflow as tf

In [2]:
file_url = 'https://storage.googleapis.com'\
           '/mledu-datasets/cats_and_dogs_filtered.zip'

In [4]:
zip_dir = tf.keras.utils.get_file('cats_and_dogs.zip', \
                                  origin=file_url, extract=True)

68616192/68606236 [==============================] - 1s 0us/step


In [5]:
import pathlib

In [6]:
path = pathlib.Path(zip_dir).parent / 'cats_and_dogs_filtered'

In [7]:
train_dir = path / 'train'
validation_dir = path / 'validation'

In [8]:
total_train = 2000
total_val = 1000

In [9]:
from tensorflow.keras.preprocessing.image \
    import ImageDataGenerator

In [10]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [11]:
batch_size=32
img_height = 224
img_width = 224

In [12]:
train_data_gen = train_image_generator.flow_from_directory\
                 (batch_size=batch_size,\
                  directory=train_dir,\
                  shuffle=True,\
                  target_size=(img_height, img_width),\
                  class_mode='binary')

Found 2000 images belonging to 2 classes.


In [13]:
val_data_gen = validation_image_generator.flow_from_directory\
               (batch_size=batch_size,\
                directory=validation_dir,\
                target_size=(img_height, img_width),\
                class_mode='binary')

Found 1000 images belonging to 2 classes.


In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [15]:
np.random.seed(8)
tf.random.set_seed(8)

In [16]:
import tensorflow_hub as hub

In [17]:
MODULE_HANDLE = 'https://tfhub.dev/google/efficientnet/b0'\
                '/feature-vector/1'
module = hub.load(MODULE_HANDLE)

In [18]:
model = tf.keras.Sequential([
        hub.KerasLayer(MODULE_HANDLE,\
                       input_shape=(224, 224, 3)),
        layers.Dense(500, activation='relu'),
        layers.Dense(1, activation='sigmoid')])

In [19]:
model.compile(loss='binary_crossentropy', \
              optimizer=tf.keras.optimizers.Adam(0.001), \
              metrics=['accuracy'])

In [20]:
model.fit(train_data_gen,\
          steps_per_epoch=total_train // batch_size,\
          epochs=5,\
          validation_data=val_data_gen,\
          validation_steps=total_val // batch_size)

Epoch 1/5
62/62 [==============================] - 28s 240ms/step - loss: 0.0554 - accuracy: 0.9776 - val_loss: 0.0247 - val_accuracy: 0.9899
Epoch 2/5
62/62 [==============================] - 14s 223ms/step - loss: 0.0116 - accuracy: 0.9959 - val_loss: 0.0253 - val_accuracy: 0.9919
Epoch 3/5
62/62 [==============================] - 14s 221ms/step - loss: 0.0032 - accuracy: 0.9995 - val_loss: 0.0245 - val_accuracy: 0.9909
Epoch 4/5
62/62 [==============================] - 14s 222ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0265 - val_accuracy: 0.9919
Epoch 5/5
62/62 [==============================] - 14s 222ms/step - loss: 5.1736e-04 - accuracy: 1.0000 - val_loss: 0.0243 - val_accuracy: 0.9929
